In [54]:
import zmlp
from zmlp import app_from_env
from zvi.display import show_asset, show_thumbnails

import os
import re
import pprint
import glob
from enum import Enum
from typing import List

import urllib3
urllib3.disable_warnings()

In [ ]:
# API Key for dev server

%env BOONAI_SERVER https://dev.api.zvi.zorroa.com/
%env BOONAI_APIKEY "PUT ENCODED API KEY HERE"

In [60]:
app = app_from_env()

In [ ]:
# Regexp you might need

letters_only = r'[^a-zA-Z]+'
numbers_only = r'[^0-9]+'

# Get Model Name

In [ ]:
model_name = "CHANGE THIS TO MODEL NAME"
model = app.models.find_one_model(name=model_name)

## Create model if model does not exist
<b>SKIP this section if you already have an existing model</b>

In [ ]:
# Reference for ModelTypes
class ModelType(Enum):
    """
    Types of models that can be Trained.
    """

    ZVI_KNN_CLASSIFIER = 0
    """A KMeans clustering model for quickly clustering assets into general groups."""

    ZVI_LABEL_DETECTION = 1
    """Retrain the ResNet50 convolutional neural network with your own labels."""

    ZVI_FACE_RECOGNITION = 2
    """Face Recognition model using a KNN classifier."""

    GCP_LABEL_DETECTION = 4
    """Train a Google AutoML vision model."""

In [ ]:
model = app.models.create_model(name=model_name, type=ModelType.ZVI_KNN_CLASSIFIER)

# Updating labels for an asset
Below are 2 functions for either deleting a single asset or batch labeling assets.

For add(), names is a List (see example) and the query searches for filenames with the name element within the filename. <b>This query may need to be updated depending on your search</b>. From here it creates a Label of that name element and adds a Label to a batch of assets matching the query.

In [ ]:
batch_max = 360  # max batch value

def delete(job_id: str = ''):
    """ Delete an asset from a query based on Job ID
    
    Args:
        job_id: (str) the Job ID which added the asset
    
    Returns:
        None
    """
    q = {
        "size": batch_max,
        "query": {
            "match": {
                "system.jobId": {
                    "query": job_id
                },
            }
        }
    }

    search = app.assets.search(q)
    for asset in search:
        print("deleting {}".format(asset.document['source']['filename']))
        app.assets.delete_asset(asset)


def add(names: List[str] = None):
    """ Batch add a label to a query of assets
    
    Args:
        names: (List[str]) list of strings to iterate through for batch labeling
        
    Returns:
        None
    """
    for name in names:
        q = {
            "size": batch_max,
            "query": {
                "match": {
                    "source.filename.fulltext": {
                        "query": name
                    },
                }
            }
        }

        label = model.make_label(name)

        search = app.assets.search(q).batches_of(batch_max)
        for batch in search:
            print("labeling {} as {}".format(len(batch), label.label))
            app.assets.update_labels(batch, label)


# Examples
# add(names=['rain', 'shine', 'sunrise', 'foggy', 'cloudy'])
# delete(job_id="e5431800-d6b7-11ea-8cb3-22473eba6bf5")


# Train Model
The following cells kick of a training job on the same `model_name` specified above.

In [25]:
model.id

'8fdc3acb-2a3c-1745-947b-c2727ff810e9'

In [12]:
app.models.train_model(model=model, deploy=True)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dev.api.zvi.zorroa.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
